In [5]:
from bs4 import BeautifulSoup
import requests
from IPython.core.display import display, HTML
import pandas as pd
import numpy as np

In [6]:
import time, os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [7]:
from bs4 import BeautifulSoup
import requests
import time, os

### Code to Pull Individual Product URLs from Wrinkle Cream Page

In [ ]:
#Scroll through entire Sephora pop-up page here to ensure all products download 
driver = webdriver.Chrome(chromedriver)
driver.get('https://www.sephora.com/shop/moisturizer-skincare?ref=100039&pageSize=300')
time.sleep(10)

In [17]:
url = 'https://www.sephora.com/shop/moisturizer-skincare?ref=100039&pageSize=300' 

response = requests.get(url)

In [18]:
response.status_code

200

In [19]:
page = response.text

In [20]:
soup = BeautifulSoup(page, "lxml")

In [21]:
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [22]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [23]:
#Pull URL for each anti-wrinkle product
value = []
for link in soup.find_all('a', {'class': 'css-ix8km1'}):
    value.append((link.get('href')))

In [24]:
#Should be around 300, not 12
len(value)

291

In [25]:
#Pull product name/brand for each anti-wrinkle product
key = []
for link in soup.find_all('a', {'class': 'css-ix8km1'}):
    key.append((link.get('aria-label')))

In [26]:
#ONLY RUN ONCE! 
#Complete URL for use in loop below to pull product info
a = 'https://www.sephora.com/'
value = [a + x for x in value] 

In [27]:
print(value[0:1])

['https://www.sephora.com//product/protini-tm-polypeptide-cream-P427421?icid2=products grid:p427421']


### Code for Pulling Specific Product Attributes

In [ ]:
#Sephora Page with Anti-Wrinkle Creams
url = 'https://www.sephora.com/shop/moisturizer-skincare?ref=100039&pageSize=300' 

response = requests.get(url)

In [ ]:
response.status_code

In [ ]:
page = response.text

In [ ]:
soup = BeautifulSoup(page, "lxml")

In [ ]:
#Don't run- long text
print(soup.find(class_= 'css-79elbk').prettify())

In [ ]:
print(soup.find(class_= 'css-79elbk').find_all('span', class_ ="css-pelz90"))

In [ ]:
#Product Brand
soup.find(class_='css-79elbk').find_all('span', class_='css-ktoumz')[0].text

In [ ]:
#Product Price
soup.find(class_= 'css-79elbk').find_all('span', class_ ="css-0")[0].text

In [ ]:
#Links to Specific Products
value = []
for link in soup.find_all('a', {'class': 'css-ix8km1'}):
    value.append((link.get('href')))

print(value)    

In [ ]:
#Drunk Elephant Polypeptide Moisturizer
url = 'https://www.sephora.com/product/protini-tm-polypeptide-cream-P427421?icid2=products%20grid:p427421' 

response = requests.get(url)

In [ ]:
response.status_code

In [ ]:
page1 = response.text

In [ ]:
soup = BeautifulSoup(page1, "lxml")

In [ ]:
#Brand
soup.find_all('span', {'class': 'css-euydo4'})[0].text

In [ ]:
#Product Name
soup.find_all('span', {'class': 'css-0'})[0].text

In [ ]:
# Number of reviews for product
soup.find_all('span', {'class': 'css-2rg6q7'})[0].text

In [ ]:
# Number of 'Loves'
soup.find('span', {'data-at': 'product_love_count'}).text

In [ ]:
#Product Size
soup.find('div', {'class': 'css-v7k1z0'}).text

In [ ]:
#Product Ingredients
#Clean up in dataframe EDA
soup.find('div', {'id': 'tabpanel2'})

In [ ]:
#Tatcha The Water Cream
url = 'https://www.sephora.com/product/the-water-cream-P418218?icid2=products%20grid:p418218'

response = requests.get(url)

In [ ]:
response.status_code  

In [ ]:
page2 = response.text

In [ ]:
soup = BeautifulSoup(page2, "lxml")

In [ ]:
soup.find_all('span', {'class': 'css-2rg6q7'})[0].text

## Create Dataframes

### Main Dataframe

In [28]:
#Remove space in url to make it usable
values = []
for i in value:
    j = i.replace(' ','')
    values.append(j)

In [29]:
print(values[0])

https://www.sephora.com//product/protini-tm-polypeptide-cream-P427421?icid2=productsgrid:p427421


In [ ]:
#Loop to get product info for all the anti-wrinkle creams
#counter = 0
prods = []
for i in values:
    response = requests.get(i)
    page = response.text
    soup = BeautifulSoup(page, "lxml")

    product = {}
    product['Name'] = soup.find('span', {'class': 'css-0'}).text
    product['Brand'] = soup.find('span', {'class': 'css-euydo4'}).text
    product['Price'] = soup.find('div', {'class': 'css-slwsq8'}).text
    product['No_reviews'] = soup.find_all('span', {'class': 'css-2rg6q7'})[0].text
    product['Hearts'] = soup.find('span', {'data-at': 'product_love_count'}).text
    product['Size'] = size = soup.find('div', {'class': 'css-v7k1z0'}).text
    
    #counter += 1

    prods.append(product)
    
    #print(counter)

In [53]:
product_df = pd.DataFrame(prods)

In [54]:
product_df.dtypes

Name          object
Brand         object
Price         object
No_reviews    object
Hearts        object
Size          object
dtype: object

In [55]:
product_df.head(2)

,Name,Brand,Price,No_reviews,Hearts,Size
0,Protini™ Polypeptide Moisturizer,Drunk Elephant,$68.00,4K reviews,266164,SIZE 1.69 oz/ 50 mL•ITEM 2025633
1,The Water Cream,Tatcha,$68.00,3K reviews,240692,ITEM 1932920


In [56]:
product_df.tail(2)

,Name,Brand,Price,No_reviews,Hearts,Size
289,Resilience Lift Firming/Sculpting Face and Nec...,Estée Lauder,$82.00,3 reviews,1011,SIZE 1.7 oz•ITEM 1468024
290,GOOPGENES All-In-One Nourishing Face Cream,goop,$95.00,0 reviews,10,SIZE 1.7 oz/ 50 mL•ITEM 2380608


#### Check for Null Values in the Dataframe

In [57]:
#Check for null values
product_df.isnull()

,Name,Brand,Price,No_reviews,Hearts,Size
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,False,False,False,False
3,False,False,False,False,False,False
4,False,False,False,False,False,False
...,...,...,...,...,...,...
286,False,False,False,False,False,False
287,False,False,False,False,False,False
288,False,False,False,False,False,False
289,False,False,False,False,False,False


In [58]:
product_df.isnull().values.any()

False

In [59]:
product_df.isnull().sum()

Name          0
Brand         0
Price         0
No_reviews    0
Hearts        0
Size          0
dtype: int64

#### Clean Up Price Column

In [60]:
#Remove dollar sign from dataframe
product_df['Price']= product_df['Price'].replace({'\$': ''}, regex = True)

In [78]:
#Update Prices to remove attached sale price point
product_df.iloc[48, product_df.columns.get_loc('Price')] = 69
product_df.iloc[53, product_df.columns.get_loc('Price')] = 69
product_df.iloc[62, product_df.columns.get_loc('Price')] = 69
product_df.iloc[79, product_df.columns.get_loc('Price')] = 102
product_df.iloc[87, product_df.columns.get_loc('Price')] = 69
product_df.iloc[184, product_df.columns.get_loc('Price')] = 99
product_df.iloc[204, product_df.columns.get_loc('Price')] = 98
product_df.iloc[255, product_df.columns.get_loc('Price')] = 59


In [79]:
#Change type to Float from Object
product_df["Price"] = pd.to_numeric(product_df["Price"], downcast="float")

#### Clean Up Reviews Column

In [80]:
#Remove rewviews from No_Reviews and replace K with 000
product_df['No_reviews'] = product_df['No_reviews'].replace({'K': '000', 'reviews': '', 'review': ''}, regex = True)

In [81]:
product_df.head(3)

,Name,Brand,Price,No_reviews,Hearts,Size
0,Protini™ Polypeptide Moisturizer,Drunk Elephant,68.0,4000,266164,SIZE 1.69 oz/ 50 mL•ITEM 2025633
1,The Water Cream,Tatcha,68.0,3000,240692,ITEM 1932920
2,Crème de la Mer Moisturizer,La Mer,180.0,667,75353,ITEM 1932201


In [82]:
#Change type to Float from Object
product_df["No_reviews"] = pd.to_numeric(product_df["No_reviews"], downcast="float")

#### Clean Up Hearts Column

In [83]:
#Change type to Float from Object
product_df["Hearts"] = pd.to_numeric(product_df["Hearts"], downcast="float")

#### Clean up Size Column

In [84]:
product_df.sort_values('Size', ascending=False)

,Name,Brand,Price,No_reviews,Hearts,Size
175,Clinique iD™: Moisturizer + Concentrate for L...,CLINIQUE,39.0,6.0,376.0,SIZE 4.2 oz/ 125 mL•ITEM 2163418
54,Clinique iD™ Custom-Blend Hydrator Collection,CLINIQUE,39.0,208.0,48493.0,SIZE 4.2 oz/ 125 mL•ITEM 2163400
150,The Essential Creme Fluid,AMOREPACIFIC,98.0,203.0,9979.0,SIZE 3 oz/ 90 mL•ITEM 1930924
19,R.N.A. POWER Anti-Aging Face Cream,SK-II,235.0,135.0,13815.0,SIZE 2.7 oz/ 80 mL•ITEM 1785229
72,Rénergie Lift Multi-Action Night Cream,Lancôme,125.0,51.0,5032.0,SIZE 2.6 oz/ 76 mL•ITEM 1459163
...,...,...,...,...,...,...
148,Regenerative Anti-Aging Moisturizer,Algenist,94.0,216.0,5707.0,ITEM 1328822
103,MOISTURE BOUND Rejuvenating Crème,AMOREPACIFIC,150.0,187.0,5858.0,ITEM 1328046
29,Vitamin Enriched Face Base Priming Moisturizer,Bobbi Brown,60.0,1000.0,114482.0,ITEM 1292820
122,Hydrating Face Cream Moisturizer,Bobbi Brown,60.0,213.0,12580.0,ITEM 1287952


In [85]:
#Check unique items in Size column
product_df.Size.unique()

array(['SIZE 1.69 oz/ 50 mL•ITEM 2025633', 'ITEM 1932920', 'ITEM 1932201',
       'ITEM 1788397', 'SIZE 1.69 oz/ 50 mL•ITEM 2233849', 'ITEM 1679315',
       'ITEM 2339489', 'ITEM 2019461', 'ITEM 2316800', 'ITEM 1932235',
       'ITEM 2175560', 'SIZE 1 oz/ 30 mL•ITEM 2031466',
       'SIZE 1.7 oz/ 50 mL•ITEM 2071751', 'ITEM 1681881',
       'SIZE 1.7 oz/ 50 mL•ITEM 2354629', 'SIZE 2 oz/ 60 mL•ITEM 2088029',
       'SIZE 2 oz/ 60 mL•ITEM 1863604', 'ITEM 2117109', 'ITEM 2346245',
       'SIZE 2.7 oz/ 80 mL•ITEM 1785229', 'ITEM 1898550',
       'SIZE 1.01 oz/ 30 mL•ITEM 2382166',
       'SIZE 1.7 oz/ 50 mL•ITEM 2106078', 'ITEM 1121797',
       'SIZE 1.7 oz/ 50 mL•ITEM 2020501',
       'SIZE 1.69 oz/ 50 mL•ITEM 2105856', 'ITEM 1910546',
       'SIZE 1.7 oz/ 50 mL•ITEM 2234110', 'ITEM 1868256', 'ITEM 1292820',
       'SIZE 1.7 oz/ 50 mL•ITEM 2178226', 'ITEM 1932193',
       'SIZE 2 oz/ 60 mL•ITEM 2170827', 'ITEM 2222958', 'ITEM 1982859',
       'SIZE 1.7 oz/ 50 mL•ITEM 1939131',
       'SIZE

In [86]:
#Remove Size from Size column
product_df['Size'] = product_df['Size'].replace({'SIZE': ''}, regex=True)
product_df.head(5)

,Name,Brand,Price,No_reviews,Hearts,Size
0,Protini™ Polypeptide Moisturizer,Drunk Elephant,68.0,4000.0,266164.0,1.69 oz/ 50 mL•ITEM 2025633
1,The Water Cream,Tatcha,68.0,3000.0,240692.0,ITEM 1932920
2,Crème de la Mer Moisturizer,La Mer,180.0,667.0,75353.0,ITEM 1932201
3,Rose & Hyaluronic Acid Deep Hydration Moisturi...,Fresh,42.0,1000.0,101831.0,ITEM 1788397
4,Lala Retro™ Whipped Moisturizer with Ceramides,Drunk Elephant,60.0,1000.0,62034.0,1.69 oz/ 50 mL•ITEM 2233849


In [87]:
start, stop, step = 0, 6, 1
product_df['Size']= product_df['Size'].str.slice(start, stop, step)
product_df.head(3)

,Name,Brand,Price,No_reviews,Hearts,Size
0,Protini™ Polypeptide Moisturizer,Drunk Elephant,68.0,4000.0,266164.0,1.69
1,The Water Cream,Tatcha,68.0,3000.0,240692.0,ITEM 1
2,Crème de la Mer Moisturizer,La Mer,180.0,667.0,75353.0,ITEM 1


In [88]:
product_df.Size.unique()

array([' 1.69 ', 'ITEM 1', 'ITEM 2', ' 1 oz/', ' 1.7 o', ' 2 oz/',
       ' 2.7 o', ' 1.01 ', ' 4.2 o', 'ITEM 6', ' 2.6 o', ' 1.68 ',
       ' 1.02 ', ' 1.61 ', ' 2.2 o', ' 1.35 ', ' 2.5 o', ' 1.6 o',
       ' 3 oz/', ' 2.1 o', ' 1 oz•', 'ITEM 3', ' 2.02 ', ' 1.3 o',
       ' 0.5 o', ' 0.35 ', ' 1.5 o'], dtype=object)

In [89]:
#Remove any leftover verbage and NA's
product_df['Size'] = product_df['Size'].replace({'ITEM 1': 'NA', 'ITEM 2': 'NA', 'ITEM 6': 'NA', 'ITEM 8': 'NA',
                                                'ITEM 3': 'NA', 'oz/': '', 'o': '', 'oz•': ''}, regex=True)
product_df.head(5)

,Name,Brand,Price,No_reviews,Hearts,Size
0,Protini™ Polypeptide Moisturizer,Drunk Elephant,68.0,4000.0,266164.0,1.69
1,The Water Cream,Tatcha,68.0,3000.0,240692.0,NA
2,Crème de la Mer Moisturizer,La Mer,180.0,667.0,75353.0,NA
3,Rose & Hyaluronic Acid Deep Hydration Moisturi...,Fresh,42.0,1000.0,101831.0,NA
4,Lala Retro™ Whipped Moisturizer with Ceramides,Drunk Elephant,60.0,1000.0,62034.0,1.69


In [90]:
product_df.Size.unique()

array([' 1.69 ', 'NA', ' 1 ', ' 1.7 ', ' 2 ', ' 2.7 ', ' 1.01 ', ' 4.2 ',
       ' 2.6 ', ' 1.68 ', ' 1.02 ', ' 1.61 ', ' 2.2 ', ' 1.35 ', ' 2.5 ',
       ' 1.6 ', ' 3 ', ' 2.1 ', ' 1 z•', ' 2.02 ', ' 1.3 ', ' 0.5 ',
       ' 0.35 ', ' 1.5 '], dtype=object)

In [91]:
product_df['Size'] = product_df['Size'].replace({'z•': ''}, regex=True)
product_df.Size.unique()

array([' 1.69 ', 'NA', ' 1 ', ' 1.7 ', ' 2 ', ' 2.7 ', ' 1.01 ', ' 4.2 ',
       ' 2.6 ', ' 1.68 ', ' 1.02 ', ' 1.61 ', ' 2.2 ', ' 1.35 ', ' 2.5 ',
       ' 1.6 ', ' 3 ', ' 2.1 ', ' 2.02 ', ' 1.3 ', ' 0.5 ', ' 0.35 ',
       ' 1.5 '], dtype=object)

In [113]:
#DO NOT RUN
product_median = product_df['Size'].median()
product_mean = product_df['Size'].mean()
print("The median of Size is", product_median)
print("The mean of Size is", product_mean)

The median of Size is 1.7000000476837158
The mean of Size is 1.7122983932495117


In [92]:
#Replace NA
product_df['Size'] = product_df['Size'].replace({'NA': '1.7'}, regex=True)
product_df.head(5)

,Name,Brand,Price,No_reviews,Hearts,Size
0,Protini™ Polypeptide Moisturizer,Drunk Elephant,68.0,4000.0,266164.0,1.69
1,The Water Cream,Tatcha,68.0,3000.0,240692.0,1.7
2,Crème de la Mer Moisturizer,La Mer,180.0,667.0,75353.0,1.7
3,Rose & Hyaluronic Acid Deep Hydration Moisturi...,Fresh,42.0,1000.0,101831.0,1.7
4,Lala Retro™ Whipped Moisturizer with Ceramides,Drunk Elephant,60.0,1000.0,62034.0,1.69


In [93]:
#Change type to Float from Object
product_df["Size"] = pd.to_numeric(product_df["Size"], downcast="float")

In [94]:
product_df.dtypes

Name           object
Brand          object
Price         float32
No_reviews    float32
Hearts        float32
Size          float32
dtype: object

### Add Column with Parent Company

In [105]:
#Add new column with name of the parent company or venture capital owner of the brand, if no pc/vc than independendent
product_df.loc[product_df['Brand'] == 'tarte', 'PC'] = 'Kose'  

product_df.loc[product_df['Brand'] == 'Bobbi Brown', 'PC'] = 'Estee_Lauder' 
product_df.loc[product_df['Brand'] == 'CLINIQUE', 'PC'] = 'Estee_Lauder' 
product_df.loc[product_df['Brand'] == 'Estée Lauder', 'PC'] = 'Estee_Lauder' 
product_df.loc[product_df['Brand'] == 'La Mer', 'PC'] = 'Estee_Lauder' 
product_df.loc[product_df['Brand'] == 'Origins', 'PC'] = 'Estee_Lauder' 
product_df.loc[product_df['Brand'] == 'Smashbox', 'PC'] = 'Estee_Lauder' 
product_df.loc[product_df['Brand'] == 'The Ordinary', 'PC'] = 'Estee_Lauder' 

product_df.loc[product_df['Brand'] == 'IT Cosmetics', 'PC'] = 'Loreal' 
product_df.loc[product_df['Brand'] == "Kiehl's Since 1851", 'PC'] = 'Loreal' 
product_df.loc[product_df['Brand'] == 'Lancôme', 'PC'] = 'Loreal' 
product_df.loc[product_df['Brand'] == 'Yves Saint Laurent', 'PC'] = 'Loreal' 

product_df.loc[product_df['Brand'] == 'Dior', 'PC'] = 'LVMH'
product_df.loc[product_df['Brand'] == 'Fresh', 'PC'] = 'LVMH'
product_df.loc[product_df['Brand'] == 'Guerlain', 'PC'] = 'LVMH'
product_df.loc[product_df['Brand'] == 'KENZOKI', 'PC'] = 'LVMH'
product_df.loc[product_df['Brand'] == 'Marc Jacobs Beauty', 'PC'] = 'LVMH'
product_df.loc[product_df['Brand'] == 'OLEHENRIKSEN', 'PC'] = 'LVMH'

product_df.loc[product_df['Brand'] == 'AMOREPACIFIC', 'PC'] = 'AMOREPACIFIC'
product_df.loc[product_df['Brand'] == 'Sulwhasoo', 'PC'] = 'AMOREPACIFIC'
product_df.loc[product_df['Brand'] == 'innisfree', 'PC'] = 'AMOREPACIFIC'

product_df.loc[product_df['Brand'] == 'philosophy', 'PC'] = 'Coty'

product_df.loc[product_df['Brand'] == 'Algenist', 'PC'] = 'PE'
product_df.loc[product_df['Brand'] == 'Obagi Clinical', 'PC'] = 'PE'
product_df.loc[product_df['Brand'] == 'Perricone MD', 'PC'] = 'PE'

product_df.loc[product_df['Brand'] == 'COOLA', 'PC'] = 'SCJ'

product_df.loc[product_df['Brand'] == 'Charlotte Tilbury', 'PC'] = 'Puig'

product_df.loc[product_df['Brand'] == 'Clarins', 'PC'] = 'Clarins'

product_df.loc[product_df['Brand'] == 'Dermalogica', 'PC'] = 'Unilever'
product_df.loc[product_df['Brand'] == "Dr Roebuck's", 'PC'] = 'Unilever'
product_df.loc[product_df['Brand'] == 'Kate Somerville', 'PC'] = 'Unilever'
product_df.loc[product_df['Brand'] == 'Tatcha', 'PC'] = 'Unilever'
product_df.loc[product_df['Brand'] == 'REN Clean Skincare', 'PC'] = 'Unilever'
product_df.loc[product_df['Brand'] == 'Murad', 'PC'] = 'Unilever'

product_df.loc[product_df['Brand'] == 'belif', 'PC'] = 'LG'

product_df.loc[product_df['Brand'] == 'First Aid Beauty', 'PC'] = 'PG'
product_df.loc[product_df['Brand'] == 'SK-II', 'PC'] = 'PG'

product_df.loc[product_df['Brand'] == 'Eve Lom', 'PC'] = 'SpaceNK'

product_df.loc[product_df['Brand'] == 'Farmacy', 'PC'] = 'Mark_Veder'

product_df.loc[product_df['Brand'] == 'Jack Black', 'PC'] = 'Edgewell'

product_df.loc[product_df['Brand'] == 'Jack Black', 'PC'] = 'Edgewell'

product_df.loc[product_df['Brand'] == 'Drunk Elephant', 'PC'] = 'Shiseido' 
product_df.loc[product_df['Brand'] == 'Shiseido', 'PC'] = 'Shiseido' 

product_df.loc[product_df['Brand'] == 'Anthony', 'PC'] = 'Independent'
product_df.loc[product_df['Brand'] == 'BeautyBio', 'PC'] = 'Independent'
product_df.loc[product_df['Brand'] == 'Beautycounter', 'PC'] = 'Independent'
product_df.loc[product_df['Brand'] == 'Biossance', 'PC'] = 'Independent'
product_df.loc[product_df['Brand'] == 'Caudalie', 'PC'] = 'Independent'
product_df.loc[product_df['Brand'] == "Dr. Barbara Sturm", 'PC'] = 'Independent'
product_df.loc[product_df['Brand'] == "Dr. Brandt Skincare", 'PC'] = 'Independent'
product_df.loc[product_df['Brand'] == 'Dr. Dennis Gross Skincare', 'PC'] = 'Independent'
product_df.loc[product_df['Brand'] == 'Edible Beauty', 'PC'] = 'Independent'
product_df.loc[product_df['Brand'] == 'FARSÁLI', 'PC'] = 'Independent'
product_df.loc[product_df['Brand'] == 'Herbivore', 'PC'] = 'Independent'
product_df.loc[product_df['Brand'] == 'Josie Maran', 'PC'] = 'Independent'
product_df.loc[product_df['Brand'] == 'KORA Organics', 'PC'] = 'Independent'
product_df.loc[product_df['Brand'] == 'KORRES', 'PC'] = 'Independent'
product_df.loc[product_df['Brand'] == 'Kopari', 'PC'] = 'Independent'
product_df.loc[product_df['Brand'] == "L'Occitane", 'PC'] = 'Independent'
product_df.loc[product_df['Brand'] == 'LANCER Skincare', 'PC'] = 'Independent'
product_df.loc[product_df['Brand'] == 'LXMI', 'PC'] = 'Independent'
product_df.loc[product_df['Brand'] == 'Moon Juice', 'PC'] = 'Independent'
product_df.loc[product_df['Brand'] == 'Naturally Serious', 'PC'] = 'Independent'
product_df.loc[product_df['Brand'] == 'Omorovicza', 'PC'] = 'Independent'
product_df.loc[product_df['Brand'] == 'Peter Thomas Roth', 'PC'] = 'Independent'
product_df.loc[product_df['Brand'] == 'Prima', 'PC'] = 'Independent'
product_df.loc[product_df['Brand'] == 'SOBEL SKIN Rx', 'PC'] = 'Independent'
product_df.loc[product_df['Brand'] == 'SUNDAY RILEY', 'PC'] = 'Independent'
product_df.loc[product_df['Brand'] == 'Saturday Skin', 'PC'] = 'Independent'
product_df.loc[product_df['Brand'] == 'Skinfix', 'PC'] = 'Independent'
product_df.loc[product_df['Brand'] == 'Supergoop!', 'PC'] = 'Independent'
product_df.loc[product_df['Brand'] == 'Tata Harper', 'PC'] = 'Independent'
product_df.loc[product_df['Brand'] == 'The INKEY List', 'PC'] = 'Independent'
product_df.loc[product_df['Brand'] == 'Volition Beauty', 'PC'] = 'Independent'
product_df.loc[product_df['Brand'] == 'Wander Beauty', 'PC'] = 'Independent'
product_df.loc[product_df['Brand'] == 'Youth To The People', 'PC'] = 'Independent'
product_df.loc[product_df['Brand'] == 'alpyn beauty', 'PC'] = 'Independent'
product_df.loc[product_df['Brand'] == 'boscia', 'PC'] = 'Independent'
product_df.loc[product_df['Brand'] == 'goop', 'PC'] = 'Independent'


product_df.head(5)

,Name,Brand,Price,No_reviews,Hearts,Size,PC
0,Protini™ Polypeptide Moisturizer,Drunk Elephant,68,4000,266164,1.69,Shiseido
1,The Water Cream,Tatcha,68,3000,240692,1.7,Unilever
2,Crème de la Mer Moisturizer,La Mer,180,667,75353,1.7,Estee_Lauder
3,Rose & Hyaluronic Acid Deep Hydration Moisturi...,Fresh,42,1000,101831,1.7,LVMH
4,Lala Retro™ Whipped Moisturizer with Ceramides,Drunk Elephant,60,1000,62034,1.69,Shiseido


### Ingredients Dataframe

#### AHA

In [106]:
AHA_df = pd.read_csv("AHA.csv")

In [107]:
AHA_df.tail(3)

,Unnamed: 0,Name,Ingredients
26,26,Clarifying Face Cream,AHA
27,27,Clinique Smart Clinical™ MD Multi-Dimensional ...,AHA
28,28,Crème du Nil Pore-Refining Moisture Veil,AHA


In [108]:
del AHA_df['Unnamed: 0']

#### Hyaluronic Acid

In [109]:
HA_df = pd.read_csv("HA.csv")
HA_df.tail(3)

,Name,Ingredients
123,Extra–Firming Day Cream All Skin Types,HA
124,Illuminating Anti-Aging Moisturizer,HA
125,Resilience Lift Firming/Sculpting Face and Nec...,HA


In [110]:
len(HA_df)

126

#### Merge Ingredients Dataframe with Products Dataframe

In [111]:
frames = [AHA_df, HA_df]
ingredients_df = pd.concat(frames)
ingredients_df.head(10)

,Name,Ingredients
0,Luna Retinol Sleeping Night Oil,AHA
1,Goodnight Glow Retin-ALT Sleeping Crème,AHA
2,C.E.O Vitamin C Rich Hydration Cream,AHA
3,Alpha Beta® Exfoliating Moisturizer,AHA
4,Ferulic + Retinol Anti-Aging Moisturizer,AHA
5,Essential-C Day Moisture Broad Spectrum SPF 30...,AHA
6,Renewed Hope in A Jar Facial Moisturizer,AHA
7,High Potency Classics: Hyaluronic Intensive Mo...,AHA
8,10% Glycolic Solutions Moisturizer,AHA
9,Hyaluronic Marine Oil-Free Moisture Cushion,AHA


In [112]:
len(ingredients_df)

155

In [113]:
ingredients_df.Ingredients.unique()

array(['AHA', 'HA'], dtype=object)

In [114]:
product_df.head(2)

,Name,Brand,Price,No_reviews,Hearts,Size,PC
0,Protini™ Polypeptide Moisturizer,Drunk Elephant,68,4000,266164,1.69,Shiseido
1,The Water Cream,Tatcha,68,3000,240692,1.7,Unilever


In [115]:
product_df['Ingredients'] = pd.Series()

In [116]:
product_df.head(2)

,Name,Brand,Price,No_reviews,Hearts,Size,PC,Ingredients
0,Protini™ Polypeptide Moisturizer,Drunk Elephant,68,4000,266164,1.69,Shiseido,NaN
1,The Water Cream,Tatcha,68,3000,240692,1.7,Unilever,NaN


In [117]:
products_df = pd.merge(left=product_df, right=ingredients_df, how='left', left_on='Name', right_on='Name')
products_df.head(5)

,Name,Brand,Price,No_reviews,Hearts,Size,PC,Ingredients_x,Ingredients_y
0,Protini™ Polypeptide Moisturizer,Drunk Elephant,68,4000,266164,1.69,Shiseido,NaN,HA
1,The Water Cream,Tatcha,68,3000,240692,1.7,Unilever,NaN,NaN
2,Crème de la Mer Moisturizer,La Mer,180,667,75353,1.7,Estee_Lauder,NaN,NaN
3,Rose & Hyaluronic Acid Deep Hydration Moisturi...,Fresh,42,1000,101831,1.7,LVMH,NaN,HA
4,Lala Retro™ Whipped Moisturizer with Ceramides,Drunk Elephant,60,1000,62034,1.69,Shiseido,NaN,HA


In [118]:
products_df.Ingredients_x.unique()

array([nan])

In [119]:
products_df.Ingredients_y.unique()

array(['HA', nan, 'AHA'], dtype=object)

In [120]:
products_df.head(2)

,Name,Brand,Price,No_reviews,Hearts,Size,PC,Ingredients_x,Ingredients_y
0,Protini™ Polypeptide Moisturizer,Drunk Elephant,68,4000,266164,1.69,Shiseido,NaN,HA
1,The Water Cream,Tatcha,68,3000,240692,1.7,Unilever,NaN,NaN


In [121]:
del products_df['Ingredients_x']

In [122]:
products_df.head(3)

,Name,Brand,Price,No_reviews,Hearts,Size,PC,Ingredients_y
0,Protini™ Polypeptide Moisturizer,Drunk Elephant,68,4000,266164,1.69,Shiseido,HA
1,The Water Cream,Tatcha,68,3000,240692,1.7,Unilever,NaN
2,Crème de la Mer Moisturizer,La Mer,180,667,75353,1.7,Estee_Lauder,NaN


In [123]:
products_df.Ingredients_y.unique()

array(['HA', nan, 'AHA'], dtype=object)

In [124]:
dummy_ingred = pd.get_dummies(products_df['Ingredients_y'])
dummy_ingred.head(10)

,AHA,HA
0,0,1
1,0,0
2,0,0
3,0,1
4,0,1
5,0,0
6,0,0
7,0,1
8,0,1
9,0,0


In [125]:
products_df = pd.concat([products_df, dummy_ingred], axis =1)
products_df.tail(25)

,Name,Brand,Price,No_reviews,Hearts,Size,PC,Ingredients_y,AHA,HA
280,Noni Glow Face Balm,KORA Organics,38,33,4924,0.35,Independent,NaN,0,0
281,Vitamin C Suncare Broad Spectrum SPF 30 Sunscreen,Obagi Clinical,58,55,1452,1.7,PE,NaN,0,0
282,V-Cense™ Youth Vitality Day Cream,REN Clean Skincare,54,20,1877,1.7,Unilever,NaN,0,0
283,Repairwear Laser Focus Line Smoothing Cream Br...,CLINIQUE,51,8,1142,1.7,Estee_Lauder,HA,0,1
284,Darker Skin Tones Face Cream Rich,Dr. Barbara Sturm,230,1,238,1.69,Independent,NaN,0,0
285,Future Solution LX Universal Defense Broad Spe...,Shiseido,100,1,468,1.7,Shiseido,NaN,0,0
286,Crème du Nil Pore-Refining Moisture Veil,LXMI,58,174,4484,1.7,Independent,AHA,1,0
287,Crème du Nil Pore-Refining Moisture Veil,LXMI,58,174,4484,1.7,Independent,HA,0,1
288,Full Spectrum 360° Mineral Sun Silk Moisturiz...,COOLA,42,6,771,1.5,SCJ,NaN,0,0
289,Visionnaire Advanced Multi-Correcting Cream,Lancôme,90,35,4281,1.7,Loreal,HA,0,1


In [126]:
dummy_brand = pd.get_dummies(product_df['Brand'])
dummy_brand.head(5)

,AMOREPACIFIC,Algenist,Anthony,BeautyBio,Beautycounter,Biossance,Bobbi Brown,CLINIQUE,COOLA,Caudalie,...,Wander Beauty,Youth To The People,Yves Saint Laurent,alpyn beauty,belif,boscia,goop,innisfree,philosophy,tarte
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [127]:
final_df = pd.concat([products_df, dummy_brand], axis = 1)
final_df.head(5)

,Name,Brand,Price,No_reviews,Hearts,Size,PC,Ingredients_y,AHA,HA,...,Wander Beauty,Youth To The People,Yves Saint Laurent,alpyn beauty,belif,boscia,goop,innisfree,philosophy,tarte
0,Protini™ Polypeptide Moisturizer,Drunk Elephant,68,4000,266164,1.69,Shiseido,HA,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,The Water Cream,Tatcha,68,3000,240692,1.7,Unilever,NaN,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Crème de la Mer Moisturizer,La Mer,180,667,75353,1.7,Estee_Lauder,NaN,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Rose & Hyaluronic Acid Deep Hydration Moisturi...,Fresh,42,1000,101831,1.7,LVMH,HA,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Lala Retro™ Whipped Moisturizer with Ceramides,Drunk Elephant,60,1000,62034,1.69,Shiseido,HA,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [129]:
dummy_product = pd.get_dummies(product_df['PC'])
dummy_product.head(5)

,AMOREPACIFIC,Clarins,Coty,Edgewell,Estee_Lauder,Independent,Kose,LG,LVMH,Loreal,Mark_Veder,PE,PG,Puig,SCJ,Shiseido,SpaceNK,Unilever
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [130]:
final_df = pd.concat([products_df, dummy_product], axis = 1)
final_df.head(5)

,Name,Brand,Price,No_reviews,Hearts,Size,PC,Ingredients_y,AHA,HA,...,LVMH,Loreal,Mark_Veder,PE,PG,Puig,SCJ,Shiseido,SpaceNK,Unilever
0,Protini™ Polypeptide Moisturizer,Drunk Elephant,68,4000,266164,1.69,Shiseido,HA,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,The Water Cream,Tatcha,68,3000,240692,1.7,Unilever,NaN,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,Crème de la Mer Moisturizer,La Mer,180,667,75353,1.7,Estee_Lauder,NaN,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Rose & Hyaluronic Acid Deep Hydration Moisturi...,Fresh,42,1000,101831,1.7,LVMH,HA,0,1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Lala Retro™ Whipped Moisturizer with Ceramides,Drunk Elephant,60,1000,62034,1.69,Shiseido,HA,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [131]:
product_df.isnull().sum()

Name             0
Brand            0
Price            0
No_reviews       0
Hearts           0
Size             0
PC               0
Ingredients    291
dtype: int64

In [134]:
final_df.to_csv(r'C:\Users\krystynametcalf\Desktop\FINALP_df.csv', index=False) 